# Part 1

In [2]:
import numpy as np
import pandas as pd
from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import normalize

In [14]:
??pd.core.dtypes.common._is_dtype

In [1]:
def get_sample(df,n):
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

def imae(x,y): return (abs(x-y)).mean()

def split_vals(a,n): return a[:n].copy(), a[n:].copy()

def print_score(m):
    maer = [imae(m.predict(X_train), y_train), imae(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): maer.append(m.oob_score_)
    print(maer)

def numericalize(df, col, name, max_n_cat):
    if not  pd.api.types.is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

def fix_missing(df, col, name, na_dict):
    if  pd.api.types.is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not pd.api.types.is_numeric_dtype(df[y_fld]): df[y_fld] = pd.Categorical(df[y_fld]).codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

TRAIN_FILE='/home/thiago.martinelli/code/main/data/pubg/train_V2.csv'
TEST_FILE='/home/thiago.martinelli/code/main/data/pubg/test_V2.csv'

df_raw = pd.read_csv(TRAIN_FILE, engine='python')

train_cats(df_raw)

df1 = df_raw.set_index('matchId',drop=False)

df2 = df1.sort_index()

df_tr, _ , _= proc_df(df2, subset = 100000)

df2.index.value_counts()

df3 = df2.iloc[:55500,:]

matches=dict(df3.index.value_counts())

m=[]
for i in matches:
    if matches[i]>=90:
        m.append(i)
len(m)

for match in set(df3.index):
    if match not in m:
        print(match)
        df4=df3.drop(match,axis=0)

dfall = df_raw.set_index(['matchId'],drop=False)

%time dfa, _, _ = proc_df(df4)

mt = dfa.copy()

cols = list(['assists', 'boosts', 'damageDealt', 'DBNOs', \
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', \
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace', \
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills', \
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', \
       'weaponsAcquired', 'winPoints'])


mt2 = mt.copy()
mt2.set_index('matchId', inplace = True)
#mt2.head(8)

mt2['mid'] = mt2.index

for c in cols:
    x = np.asarray(mt2.groupby('mid')[c].apply(lambda x: normalize(np.asarray(x).reshape(1,-1))))
    l=[]
    for i in x:
        for j in i:
            l.extend(j)
    mt2.insert(value = l, column = 'pos_'+c, loc=0)
    

newdf=mt2.copy()

newdf = newdf.iloc[:: -1]

df_tr = newdf.drop(['winPlacePerc'],axis=1)
y_tr = newdf['winPlacePerc']

n_valid = 1000  # same as Kaggle's test set size
n_tr = len(df_tr)-n_valid
#raw_train, raw_valid = split_vals(df_raw, n_tr)
X_train, X_valid = split_vals(df_tr, n_tr)
y_train, y_valid = split_vals(y_tr, n_tr)
X_train.shape, y_train.shape, X_valid.shape

m = RandomForestRegressor(n_jobs=-1, n_estimators = 50 )
m.fit(X_train, y_train)
print_score(m)

In [1]:
?train_cats

Object `train_cats` not found.
